In [17]:
import tensorflow as tf

import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import pyod

from sklearn.neighbors import kneighbors_graph
from collections import Counter

In [3]:
root_path="../Projectors_publication/projector_Conso_CVAE_5couches-Day_WorkingDays_L1/"
sess= tf.Session()
saver = tf.train.import_meta_graph(root_path + "tf_data.ckpt.meta")
saver.restore(sess,root_path+"tf_data.ckpt")

INFO:tensorflow:Restoring parameters from ../Projectors_publication/projector_Conso_CVAE_5couches-Day_WorkingDays_L1/tf_data.ckpt


In [4]:
tf.get_variable('Variable_2', shape=(1500, 4))
embedding = sess.run('Variable_2:0')

In [5]:
embedding.shape

(1500, 4)

In [6]:
from __future__ import division
from __future__ import print_function

import os
import sys
from time import time

# temporary solution for relative imports in case pyod is not installed
# if pyod is installed, no need to use the following line
sys.path.append(
    os.path.abspath(os.path.join(os.path.dirname("__file__"), '..')))
# supress warnings for clean output
import warnings

warnings.filterwarnings("ignore")


from sklearn.model_selection import train_test_split
from scipy.io import loadmat

from pyod.models.abod import ABOD
from pyod.models.cblof import CBLOF
from pyod.models.feature_bagging import FeatureBagging
from pyod.models.hbos import HBOS
from pyod.models.iforest import IForest
from pyod.models.knn import KNN
from pyod.models.lof import LOF
from pyod.models.mcd import MCD
from pyod.models.ocsvm import OCSVM
from pyod.models.pca import PCA
from pyod.models.lscp import LSCP

from pyod.utils.utility import standardizer
from pyod.utils.utility import precision_n_scores
from sklearn.metrics import roc_auc_score

In [23]:
# TODO: add neural networks, LOCI, SOS
# TODO: and update output precision (=4)
# Define data file and read X and y

# 60% data for training and 40% for testing
embedding_norm = standardizer(embedding)
random_state = np.random.RandomState(42)
outliers_fraction = 20/1000
outliers_percentage = round(outliers_fraction * 100, ndigits=4)
# initialize a set of detectors for LSCP
detector_list = [LOF(n_neighbors=10), LOF(n_neighbors=20), LOF(n_neighbors=30),
                 LOF(n_neighbors=40), LOF(n_neighbors=50)]
classifiers = {'Angle-based Outlier Detector (ABOD)': ABOD(
    contamination=outliers_fraction),
    'Cluster-based Local Outlier Factor': CBLOF(
        contamination=outliers_fraction, check_estimator=False,
        random_state=random_state),
    'Feature Bagging': FeatureBagging(contamination=outliers_fraction,
                                      check_estimator=False,
                                      random_state=random_state),
    'Histogram-base Outlier Detection (HBOS)': HBOS(
        contamination=outliers_fraction),
    'Isolation Forest': IForest(contamination=outliers_fraction,
                                random_state=random_state),
    'K Nearest Neighbors (KNN)': KNN(contamination=outliers_fraction),
    'Local Outlier Factor (LOF)': LOF(
        contamination=outliers_fraction),
    'Minimum Covariance Determinant (MCD)': MCD(
        contamination=outliers_fraction, random_state=random_state),
    'One-class SVM (OCSVM)': OCSVM(contamination=outliers_fraction,
                                   random_state=random_state),
    'Principal Component Analysis (PCA)': PCA(
        contamination=outliers_fraction, random_state=random_state),
    'Locally Selective Combination (LSCP)': LSCP(
        detector_list, contamination=outliers_fraction,
        random_state=random_state)
}
classifiers_indices = {
    'Angle-based Outlier Detector (ABOD)': 0,
    'Cluster-based Local Outlier Factor': 1,
    'Feature Bagging': 2,
    'Histogram-base Outlier Detection (HBOS)': 3,
    'Isolation Forest': 4,
    'K Nearest Neighbors (KNN)': 5,
    'Local Outlier Factor (LOF)': 6,
    'Minimum Covariance Determinant (MCD)': 7,
    'One-class SVM (OCSVM)': 8,
    'Principal Component Analysis (PCA)': 9,
    'Locally Selective Combination (LSCP)': 10}

n_ite = 50
n_classifiers = 11

df_columns = ['Data', '#Samples', '# Dimensions', 'Outlier Perc',
              'ABOD', 'CBLOF', 'FB', 'HBOS', 'IForest', 'KNN', 'LOF',
              'MCD', 'OCSVM', 'PCA', 'LSCP']
time_df = pd.DataFrame(columns=df_columns)



res_dict = {}
for clf_name, clf in classifiers.items():
    res_dict[clf_name]=[]

time_mat = np.zeros([n_ite, n_classifiers])
res_mat = np.zeros([n_ite, n_classifiers])


for i in range(n_ite):
    print("\n... Processing", '...', 'Iteration', i + 1)
    random_state = np.random.RandomState(i)
    outliers_fraction = 20/1000
    outliers_percentage = round(outliers_fraction * 100, ndigits=4)

    classifiers = {'Angle-based Outlier Detector (ABOD)': ABOD(
        contamination=outliers_fraction),
        'Cluster-based Local Outlier Factor': CBLOF(
            contamination=outliers_fraction, check_estimator=False,
            random_state=random_state),
        'Feature Bagging': FeatureBagging(contamination=outliers_fraction,
                                          check_estimator=False,
                                          random_state=random_state),
        'Histogram-base Outlier Detection (HBOS)': HBOS(
            contamination=outliers_fraction),
        'Isolation Forest': IForest(contamination=outliers_fraction,
                                    random_state=random_state),
        'K Nearest Neighbors (KNN)': KNN(contamination=outliers_fraction),
        'Local Outlier Factor (LOF)': LOF(
            contamination=outliers_fraction),
        'Minimum Covariance Determinant (MCD)': MCD(
            contamination=outliers_fraction, random_state=random_state),
        'One-class SVM (OCSVM)': OCSVM(contamination=outliers_fraction,
                                       random_state=random_state),
        'Principal Component Analysis (PCA)': PCA(
            contamination=outliers_fraction, random_state=random_state),

    }
    for clf_name, clf in classifiers.items():
        t0 = time()
        res =np.where(clf.fit_predict(embedding_norm) != 0)[0]
        t1 = time()
        duration = round(t1 - t0, ndigits=4)

        print('ratio for {} {} in {}'.format(clf_name,len(res),duration))

        time_mat[i, classifiers_indices[clf_name]] = duration
        res_dict[clf_name]+=list(res)



... Processing ... Iteration 1
ratio for Angle-based Outlier Detector (ABOD) 30 in 0.1904
ratio for Cluster-based Local Outlier Factor 30 in 0.087
ratio for Feature Bagging 30 in 0.0724
ratio for Histogram-base Outlier Detection (HBOS) 28 in 0.0014
ratio for Isolation Forest 30 in 0.1741
ratio for K Nearest Neighbors (KNN) 30 in 0.0048
ratio for Local Outlier Factor (LOF) 30 in 0.0074
ratio for Minimum Covariance Determinant (MCD) 30 in 0.6417
ratio for One-class SVM (OCSVM) 30 in 0.0787
ratio for Principal Component Analysis (PCA) 30 in 0.0017

... Processing ... Iteration 2
ratio for Angle-based Outlier Detector (ABOD) 30 in 0.1968
ratio for Cluster-based Local Outlier Factor 30 in 0.0775
ratio for Feature Bagging 30 in 0.0758
ratio for Histogram-base Outlier Detection (HBOS) 28 in 0.0016
ratio for Isolation Forest 30 in 0.1801
ratio for K Nearest Neighbors (KNN) 30 in 0.0049
ratio for Local Outlier Factor (LOF) 30 in 0.0073
ratio for Minimum Covariance Determinant (MCD) 30 in 0.642

ratio for Angle-based Outlier Detector (ABOD) 30 in 0.1953
ratio for Cluster-based Local Outlier Factor 30 in 0.0723
ratio for Feature Bagging 30 in 0.0717
ratio for Histogram-base Outlier Detection (HBOS) 28 in 0.0014
ratio for Isolation Forest 30 in 0.1777
ratio for K Nearest Neighbors (KNN) 30 in 0.0048
ratio for Local Outlier Factor (LOF) 30 in 0.0075
ratio for Minimum Covariance Determinant (MCD) 30 in 0.6598
ratio for One-class SVM (OCSVM) 30 in 0.0806
ratio for Principal Component Analysis (PCA) 30 in 0.0015

... Processing ... Iteration 17
ratio for Angle-based Outlier Detector (ABOD) 30 in 0.1987
ratio for Cluster-based Local Outlier Factor 30 in 0.0757
ratio for Feature Bagging 30 in 0.0714
ratio for Histogram-base Outlier Detection (HBOS) 28 in 0.0017
ratio for Isolation Forest 30 in 0.176
ratio for K Nearest Neighbors (KNN) 30 in 0.0047
ratio for Local Outlier Factor (LOF) 30 in 0.0074
ratio for Minimum Covariance Determinant (MCD) 30 in 0.6479
ratio for One-class SVM (OCSV

ratio for Angle-based Outlier Detector (ABOD) 30 in 0.2005
ratio for Cluster-based Local Outlier Factor 30 in 0.0843
ratio for Feature Bagging 30 in 0.0785
ratio for Histogram-base Outlier Detection (HBOS) 28 in 0.0016
ratio for Isolation Forest 30 in 0.2135
ratio for K Nearest Neighbors (KNN) 30 in 0.0051
ratio for Local Outlier Factor (LOF) 30 in 0.0081
ratio for Minimum Covariance Determinant (MCD) 30 in 0.7019
ratio for One-class SVM (OCSVM) 30 in 0.084
ratio for Principal Component Analysis (PCA) 30 in 0.0017

... Processing ... Iteration 32
ratio for Angle-based Outlier Detector (ABOD) 30 in 0.2028
ratio for Cluster-based Local Outlier Factor 30 in 0.0814
ratio for Feature Bagging 30 in 0.0664
ratio for Histogram-base Outlier Detection (HBOS) 28 in 0.0018
ratio for Isolation Forest 30 in 0.1756
ratio for K Nearest Neighbors (KNN) 30 in 0.0042
ratio for Local Outlier Factor (LOF) 30 in 0.0078
ratio for Minimum Covariance Determinant (MCD) 30 in 0.6591
ratio for One-class SVM (OCSV

ratio for Angle-based Outlier Detector (ABOD) 30 in 0.1759
ratio for Cluster-based Local Outlier Factor 30 in 0.0773
ratio for Feature Bagging 30 in 0.0649
ratio for Histogram-base Outlier Detection (HBOS) 28 in 0.0016
ratio for Isolation Forest 30 in 0.1599
ratio for K Nearest Neighbors (KNN) 30 in 0.0042
ratio for Local Outlier Factor (LOF) 30 in 0.0066
ratio for Minimum Covariance Determinant (MCD) 30 in 0.6232
ratio for One-class SVM (OCSVM) 30 in 0.0811
ratio for Principal Component Analysis (PCA) 30 in 0.0021

... Processing ... Iteration 47
ratio for Angle-based Outlier Detector (ABOD) 30 in 0.2183
ratio for Cluster-based Local Outlier Factor 30 in 0.0898
ratio for Feature Bagging 30 in 0.0721
ratio for Histogram-base Outlier Detection (HBOS) 28 in 0.0025
ratio for Isolation Forest 30 in 0.1779
ratio for K Nearest Neighbors (KNN) 30 in 0.0047
ratio for Local Outlier Factor (LOF) 30 in 0.0072
ratio for Minimum Covariance Determinant (MCD) 30 in 0.6073
ratio for One-class SVM (OCS

In [39]:
outliers_freq_dict=dict()
for clf_name, clf in classifiers.items():
    outliers_freq_dict[clf_name]=[]

for key, value in res_dict.items():
    outliers_freq_dict[key] = dict(Counter(sorted(value)))

In [52]:
outliers_dict=dict()
for clf_name, clf in classifiers.items():
    outliers_dict[clf_name]=[]

for key, value in outliers_freq_dict.items():
    outliers_dict[key] = [k for k,v in value.items()]

In [53]:
outliers_list = []
for key, value in outliers_dict.items():
    outliers_list +=value

In [54]:
dict(Counter(sorted(outliers_list)))


{5: 3,
 17: 4,
 18: 1,
 19: 4,
 20: 3,
 21: 7,
 22: 4,
 23: 4,
 39: 1,
 53: 1,
 57: 9,
 58: 1,
 65: 3,
 66: 7,
 74: 1,
 77: 6,
 107: 4,
 130: 1,
 170: 2,
 223: 1,
 224: 1,
 227: 2,
 228: 2,
 229: 2,
 231: 10,
 232: 5,
 240: 2,
 304: 1,
 327: 2,
 329: 1,
 435: 4,
 437: 4,
 455: 1,
 493: 1,
 497: 5,
 518: 4,
 584: 1,
 585: 1,
 586: 1,
 587: 1,
 588: 3,
 591: 3,
 592: 4,
 593: 3,
 594: 4,
 598: 1,
 599: 1,
 601: 4,
 603: 2,
 604: 2,
 668: 1,
 675: 1,
 703: 8,
 705: 2,
 730: 8,
 769: 1,
 770: 6,
 771: 1,
 775: 3,
 797: 1,
 798: 4,
 799: 1,
 815: 2,
 831: 7,
 832: 5,
 868: 1,
 928: 3,
 929: 7,
 945: 2,
 950: 8,
 951: 3,
 952: 5,
 955: 1,
 956: 9,
 957: 10,
 958: 1,
 959: 3,
 960: 1,
 962: 1,
 963: 1,
 964: 1,
 966: 2,
 967: 1,
 968: 2,
 1008: 1,
 1046: 1,
 1058: 8,
 1116: 4,
 1208: 4,
 1223: 6,
 1225: 8,
 1245: 3,
 1260: 2,
 1294: 1,
 1295: 1,
 1319: 1,
 1320: 1,
 1321: 3,
 1322: 1,
 1323: 6,
 1327: 5,
 1336: 3,
 1464: 1,
 1467: 4,
 1477: 4,
 1482: 6,
 1483: 6,
 1484: 5,
 1485: 5,
 1486: 1,

In [56]:
real_outliers =dict(Counter(sorted(outliers_list)))
np.savez('outliers_day_wd', outliers=real_outliers)

array({5: 3, 17: 4, 18: 1, 19: 4, 20: 3, 21: 7, 22: 4, 23: 4, 39: 1, 53: 1, 57: 9, 58: 1, 65: 3, 66: 7, 74: 1, 77: 6, 107: 4, 130: 1, 170: 2, 223: 1, 224: 1, 227: 2, 228: 2, 229: 2, 231: 10, 232: 5, 240: 2, 304: 1, 327: 2, 329: 1, 435: 4, 437: 4, 455: 1, 493: 1, 497: 5, 518: 4, 584: 1, 585: 1, 586: 1, 587: 1, 588: 3, 591: 3, 592: 4, 593: 3, 594: 4, 598: 1, 599: 1, 601: 4, 603: 2, 604: 2, 668: 1, 675: 1, 703: 8, 705: 2, 730: 8, 769: 1, 770: 6, 771: 1, 775: 3, 797: 1, 798: 4, 799: 1, 815: 2, 831: 7, 832: 5, 868: 1, 928: 3, 929: 7, 945: 2, 950: 8, 951: 3, 952: 5, 955: 1, 956: 9, 957: 10, 958: 1, 959: 3, 960: 1, 962: 1, 963: 1, 964: 1, 966: 2, 967: 1, 968: 2, 1008: 1, 1046: 1, 1058: 8, 1116: 4, 1208: 4, 1223: 6, 1225: 8, 1245: 3, 1260: 2, 1294: 1, 1295: 1, 1319: 1, 1320: 1, 1321: 3, 1322: 1, 1323: 6, 1327: 5, 1336: 3, 1464: 1, 1467: 4, 1477: 4, 1482: 6, 1483: 6, 1484: 5, 1485: 5, 1486: 1, 1488: 3, 1489: 7, 1490: 4},
      dtype=object)

In [57]:
len(real_outliers)

113

^(21|57|66|77|231|703|730|770|831|929|950|956|957|1058|1223|1225|1323|1482|1483|1489)$

# Outlier detection using CFOF score
Intuitively, the CFOF score measures how many neighbors have to be taken into account in order for the object to be considered close by an appreciable fraction of the dataset objects. 

In [5]:
# hyperparameters settings 
n = 50

In [6]:
neighbors_graph = dict()

for i in range(1, n): 
    print(i, end='\r')
    neighbors_graph[i] = kneighbors_graph(X=embedding, n_neighbors=i, mode='connectivity')

In [7]:
def Nk(i,matrix_k):
    return sum(neighbors_graph[matrix_k].toarray()[:,i])-1
    
def CFOF_score(x_index, rho=0.001):
    k_list = []
    for k in range(1,n): 
        if Nk(x_index,k) >=(rho * n): 
            k_list.append(k/n)
    if len(k_list) == 0:
        raise ValueError('Error ')
    else : 
        return min(k_list)
    

In [8]:
cfof_embedding = []

for i in range(len(embedding)):
    print(i, end='\r')
    cfof_embedding.append(CFOF_score(i, rho=0))
plt.figure()
plt.hist(cfof_embedding, bins = 100)
plt.show()

ValueError: Error 

In [ ]:
potential_outlier_with_threshold = np.where(np.array(cfof_embedding) > 0.1)[0]
potential_outlier_with_threshold

In [ ]:
def get_all_cfof_scores():
    cfof_thresholds = dict()
    for r in range(1,10):
        rho = r/10
        cfof_embedding = []

        for i in range(len(embedding)):
            print(i, end='\r')
            cfof_embedding.append(CFOF_score(i, rho))
        #plt.figure()
        #plt.hist(cfof_embedding, bins = 100)
        #plt.show()
        cfof_thresholds[rho] = cfof_embedding
    
    return cfof_thresholds

cfof_thresholds_dict = get_all_cfof_scores()